In [ ]:
#Text_Preprocessing에서 변수들 가져와야 함 위치에 따라서 import 시켜주세요
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import xgboost as xgb
from sklearn.metrics import accuracy_score

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

dtrain = xgb.DMatrix(X_train_vec, label=Y_train)

# XGBoost 파라미터 조절
xgb_params = {
    'objective': 'binary:logistic',  # 이진 분류
    'max_depth': 50,  # 트리 최대 깊이
    'silent': 1,  # 출력 관련 메시지
    'eta': 0.3,  # Learning rate 
    'subsample': 0.8,  # 학습 시 데이터 샘플링 비율 
    'colsample_bytree': 0.5,  # 트리 생성에 필요한 feature 샘플링에 사용
    'alpha': 0.5,  # L1 regularization weight 값
    'tree_method': 'gpu_hist'  # gpu 사용
}
# XGBoost 학습
xgb_model = xgb.train(xgb_params, dtrain, num_boost_round=150)

# model 저장
xgb_model.save_model("/content/xgb_model.model")

# vectorizer 저장
joblib.dump(vectorizer, "/content/vectorizer.pkl")

# XGBoost 모델 테스트 
X_test_vec = vectorizer.transform(X_test)
xgb_test = xgb.DMatrix(X_test_vec, label=Y_test)

xgb_y_pred = xgb_model.predict(xgb_test)
xgb_predictions = [round(value) for value in xgb_y_pred]

# 정확도 계산
xgb_accuracy = accuracy_score(Y_test, xgb_predictions)

print("XGBoost accuracy:", xgb_accuracy)